In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

In [ ]:
! rsync --progress "bluemoon-user1.uvm.edu:thesis/data/0.608-0.267.pkl" "../../data/0.608-0.267.pkl"

In [ ]:
%%time
with open("../../data/0.608-0.267.pkl", "rb") as f:
    (b, i0, x_rev, λ, θ, μ, s, x_rest, α, n1, β, n2, G1, G2), sol, phase, χ, m = pickle.load(f)

In [ ]:
title = f"alpha: {α:.03f}, beta: {β:.03f}, chi: {χ:.04f}, m: {m:.04f}"

size = (12, 6)

In [ ]:
metadata = pd.read_excel("../connectomes/mouse_meta.xlsx", sheet_name=None)

mtdt = metadata["Voxel Count_295 Structures"]
del(metadata)
mtdt = mtdt.loc[mtdt["Represented in Linear Model Matrix"] == "Yes"]

columns = []
cortices = [[0, 0]]
regions = mtdt["Major Region"].unique()
for region in regions:
    i = [columns.append(acronym.replace(" ", "")) for acronym in
         mtdt.loc[mtdt["Major Region"] == region, "Acronym"].values]
    cortices.append([cortices[-1][-1], cortices[-1][-1] + len(i)])
cortices.remove([0, 0])
del(mtdt)

In [ ]:
y = sol.y.T.reshape(phase.shape[1], 3, -1)
del(sol)

means = np.zeros([len(cortices), phase.shape[0]])
sums = np.copy(means)

for i, cortex in enumerate(cortices):
    means[i] = np.mean(y[cortex[0]:cortex[1], 0, :], axis=0)
    sums[i] = np.sum(y[cortex[0]:cortex[1], 0, :], axis=0)

for i, mn in enumerate(means):
    plt.plot(mn + 20*i, lw=0.07, color="k")

plt.xlim([0, means.shape[1]])
plt.title(title)
plt.xlabel("time")
plt.ylabel("mean x")

plt.yticks([20*i for i in range(len(regions))], regions)

fig = plt.gcf()
fig.set_size_inches(*size, forward=True)
fig.savefig(f"means-{α:.03f}-{β:.03f}.png", dpi=700,
            bbox_inches="tight", format="png")
plt.cla()
plt.clf()

for i, sm in enumerate(sums):
    plt.plot(sm + 100*i, lw=0.07, color="k")

plt.xlim([0, sums.shape[1]])
plt.title(title)
plt.xlabel("time")
plt.ylabel("sum x")

plt.yticks([100*i for i in range(len(regions))], regions)

fig = plt.gcf()
fig.set_size_inches(*size, forward=True)
fig.savefig(f"sums-{α:.03f}-{β:.03f}.png", dpi=700,
            bbox_inches="tight", format="png")
plt.cla()
plt.clf()